**Simple Generative AI App using LangChain**

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_PROJECT']=os.getenv("LANGCHAIN_PROJECT")

Data Ingestion->From the website, we need to scrape the data

In [4]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader=WebBaseLoader("https://docs.smith.langchain.com/prompt_engineering/how_to_guides/manage_prompts_programatically")
loader

In [6]:
page_docs=loader.load()
page_docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/manage_prompts_programatically', 'title': 'Manage prompts programmatically | 🦜️🛠️ LangSmith', 'description': 'You can use the LangSmith Python and TypeScript SDK to manage prompts programmatically.', 'language': 'en'}, page_content='\n\n\n\n\nManage prompts programmatically | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringQuickstartsTutorialsOptimize a classifierHow-to GuidesCreate a promptRun the playground against a custom LangServe model serverRun the playground against an OpenAI-compliant model provider/proxyUpdate a promptUse custom TLS certificatesManage prompts programmaticallyManaging Prompt SettingsPrompt TagsOpen a prompt from a traceTesting over a datasetLangChain HubPrompt CanvasConce

With different LLM models, there are limited context size, so we cannot give the entire data directly. \
Hence, we need to built chunks of texts from data

### Load Data -> Docs -> Divide our documents into chunks -> vectors using Vector Embeddings -> Vector Store DB

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

docs=splitter.split_documents(page_docs)

In [8]:
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/manage_prompts_programatically', 'title': 'Manage prompts programmatically | 🦜️🛠️ LangSmith', 'description': 'You can use the LangSmith Python and TypeScript SDK to manage prompts programmatically.', 'language': 'en'}, page_content='Manage prompts programmatically | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/manage_prompts_programatically', 'title': 'Manage prompts programmatically | 🦜️🛠️ LangSmith', 'description': 'You can use the LangSmith Python and TypeScript SDK to manage prompts programmatically.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringQuickstartsTutorialsOptimize a classifierHow-to GuidesCreate a promptRu

In [9]:
# using OpenAIEmbedding model
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

Using FAISS for storing vectorstore DB

In [10]:
from langchain_community.vectorstores import FAISS
vectorstore_db=FAISS.from_documents(docs,embeddings)

In [11]:
vectorstore_db

Query from Vectorstore DB

In [12]:
query="payload required for the OpenAI or Anthropic"
res=vectorstore_db.similarity_search(query)
res[0].page_content

'These convert your prompt into the payload required for the OpenAI or Anthropic API.\nThese conversion methods rely on logic from within LangChain integration packages, and you will need to install the appropriate package as a dependency\nin addition to your official SDK of choice. Here are some examples:\nOpenAI\u200b\nPythonTypeScriptpip install -U langchain_openaiyarn add @langchain/openai @langchain/core // @langchain/openai version >= 0.3.2'

In [13]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model='gpt-4o-mini')

Retrieval Chain, Documents Chain

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
  """
Answer the following question based only on the provided context:
<context>
{context}
</context>

"""
)

In [15]:
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001824E7D8F70>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001824E7DB040>, root_client=<openai.OpenAI object at 0x0000018255377CA0>, root_async_client=<openai.AsyncOpenAI object at 0x000001824E7D8FD0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, co

In [16]:
from langchain_core.documents import Document
document_chain.invoke({
  "input":"payload required for the OpenAI or Anthropic",
  "context":[Document(page_content="Use a prompt without LangChain. If you want to store your prompts in LangSmith but use them directly with a model provider's API, you can use our conversion methods. These convert your prompt into the payload required for the OpenAI or Anthropic API. These conversion methods rely on logic from within LangChain integration packages, and you will need to install the appropriate package as a dependency in addition to your official SDK of choice. Here are some examples:")]
})

'What is the purpose of the conversion methods mentioned in the context?'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

---
**Retriever** \
Input--->Retriever--->vectorstore_db


In [17]:
vectorstore_db

In [18]:
retriever=vectorstore_db.as_retriever()

In [19]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [20]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018255375C30>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
            | Ch

Get the response from the LLM

In [21]:
res=retrieval_chain.invoke({
  "input":"payload required for the OpenAI or Anthropic"
})
res['answer']

'What is required to use the conversion methods mentioned in the context for the OpenAI or Anthropic API?\n\nTo use the conversion methods for the OpenAI or Anthropic API, you need to install the appropriate LangChain integration packages as a dependency, in addition to your official SDK of choice.'

In [22]:
res

{'input': 'payload required for the OpenAI or Anthropic',
 'context': [Document(id='35cd6ac2-55e4-4fef-afff-1c76bc1b9556', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/manage_prompts_programatically', 'title': 'Manage prompts programmatically | 🦜️🛠️ LangSmith', 'description': 'You can use the LangSmith Python and TypeScript SDK to manage prompts programmatically.', 'language': 'en'}, page_content='These convert your prompt into the payload required for the OpenAI or Anthropic API.\nThese conversion methods rely on logic from within LangChain integration packages, and you will need to install the appropriate package as a dependency\nin addition to your official SDK of choice. Here are some examples:\nOpenAI\u200b\nPythonTypeScriptpip install -U langchain_openaiyarn add @langchain/openai @langchain/core // @langchain/openai version >= 0.3.2'),
  Document(id='189b6a70-6aed-49ec-955d-e70a1e8bf51d', metadata={'source': 'https://docs.smith.langchain.

In [23]:
res['context']

[Document(id='35cd6ac2-55e4-4fef-afff-1c76bc1b9556', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/manage_prompts_programatically', 'title': 'Manage prompts programmatically | 🦜️🛠️ LangSmith', 'description': 'You can use the LangSmith Python and TypeScript SDK to manage prompts programmatically.', 'language': 'en'}, page_content='These convert your prompt into the payload required for the OpenAI or Anthropic API.\nThese conversion methods rely on logic from within LangChain integration packages, and you will need to install the appropriate package as a dependency\nin addition to your official SDK of choice. Here are some examples:\nOpenAI\u200b\nPythonTypeScriptpip install -U langchain_openaiyarn add @langchain/openai @langchain/core // @langchain/openai version >= 0.3.2'),
 Document(id='189b6a70-6aed-49ec-955d-e70a1e8bf51d', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/manage_prompts_programatically', '